## Ez a munkalap egy audió file daraboló, ami a feltöltött hanganyagot mondatokra, rövidebb szövegrészletekre bontja.

**Függőségek telepítése**

In [ ]:
# Szükséges könyvtárak telepítése
!pip install moviepy pandas
!pip install faster-whisper
!pip install srt
!pip install pydub
# Telepítsd az ffmpeg-et
!apt-get update
!apt-get install -y ffmpeg

**MP3 fileok és hozzá tartozó whisper-large-v2-vel készített str fileok feltöltése**

In [ ]:
import os
from google.colab import files

# Mappa létrehozása, ha még nem létezik
os.makedirs('/content/files', exist_ok=True)

# Fájlok feltöltése
uploaded = files.upload()

# Fájlok áthelyezése a /content/files mappába
for file_name in uploaded.keys():
    os.rename(file_name, f'/content/files/{file_name}')

print("Fájlok sikeresen feltöltve és áthelyezve a /content/files mappába.")

Felirat file(ok) készítése (ha töltöttél fel az MP3-ak mellé, akkor kihagyható)

In [ ]:
import os
import srt
import torch
from faster_whisper import WhisperModel
from datetime import timedelta

# Inicializáljuk a modellt
model = WhisperModel("large-v2", device="cuda" if torch.cuda.is_available() else "cpu")

# Beállítjuk a könyvtárat
directory = "/content/files"

# Létrehozzuk az SRT fájlokat
def seconds_to_timedelta(seconds):
    return timedelta(seconds=seconds)

for filename in os.listdir(directory):
    if filename.endswith(".mp3") or filename.endswith(".wav") or filename.endswith(".m4a"):  # kiterjesztés alapján szűrünk
        file_path = os.path.join(directory, filename)
        print(f"Processing {file_path}")

        # Beolvasás és transzkripció
        segments, info = model.transcribe(file_path, language="hu")

        # SRT fájl létrehozása
        srt_filename = os.path.splitext(file_path)[0] + ".srt"
        with open(srt_filename, "w", encoding="utf-8") as srt_file:
            subs = []
            for i, segment in enumerate(segments):
                start = seconds_to_timedelta(segment.start)
                end = seconds_to_timedelta(segment.end)
                text = segment.text
                subs.append(srt.Subtitle(index=i, start=start, end=end, content=text))
            srt_file.write(srt.compose(subs))

print("Transcription and SRT generation completed.")


A files mappába feltöltött mp3+str párosok darabolása

In [ ]:
import pandas as pd
import re
import os
import subprocess

# SRT fájl beolvasása
def parse_srt(srt_path):
    with open(srt_path, 'r', encoding='utf-8') as file:
        srt_data = file.read()

    pattern = re.compile(r'(\d+)\s+(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\s+(.+?)\s*(?=\d+\s+\d{2}:\d{2}:\d{2},\d{3} -->|\Z)', re.S)
    matches = pattern.findall(srt_data)

    srt_entries = []
    for match in matches:
        start_time = match[1].replace(',', '.')
        end_time = match[2].replace(',', '.')
        text = match[3].strip()
        srt_entries.append((start_time, end_time, text))

    return pd.DataFrame(srt_entries, columns=['start', 'end', 'text'])

# Időbélyeg konvertálása másodpercekre
def time_to_seconds(time_str):
    h, m, s = map(float, time_str.split(':'))
    return h * 3600 + m * 60 + s

# Hangfájl feldarabolása
def split_audio(audio_path, srt_df, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Az eredeti fájl nevének kinyerése kiterjesztés nélkül
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for index, row in srt_df.iterrows():
        start_time = time_to_seconds(row['start'])
        end_time = time_to_seconds(row['end'])
        duration = end_time - start_time

        print(f"Processing clip {index + 1} from {start_time} to {end_time}")

        output_file = os.path.join(output_folder, f"{base_name}_{index + 1}.mp3")

        # FFmpeg parancs futtatása
        command = [
            'ffmpeg', '-y', '-i', audio_path, '-ss', str(start_time), '-t', str(duration),
            '-acodec', 'copy', output_file
        ]

        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

       # print(f"Command: {' '.join(command)}")
       # print(f"stdout: {result.stdout.decode('utf-8')}")
       # print(f"stderr: {result.stderr.decode('utf-8')}")
       # print(f"Saved clip to {output_file}")

# Fő program
def process_directory(directory_path, output_folder):
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.mp3'):
            audio_path = os.path.join(directory_path, file_name)
            srt_path = os.path.splitext(audio_path)[0] + '.srt'
            if os.path.exists(srt_path):
                print(f"Processing pair: {audio_path} and {srt_path}")
                srt_df = parse_srt(srt_path)
                split_audio(audio_path, srt_df, output_folder)
            else:
                print(f"No matching SRT file for {audio_path}")

# Add meg a könyvtár pontos útvonalát
directory_path = '/content/files'
output_folder = '/content/splits'

process_directory(directory_path, output_folder)

In [ ]:
from faster_whisper import WhisperModel
import os
import torch

# Initialize the model
model_size = "large-v2"  # You can choose other model sizes as needed
model = WhisperModel(model_size, device="cuda" if torch.cuda.is_available() else "cpu")

# Directory containing audio files
input_dir = "/content/splits"

# Process each file in the directory
for filename in os.listdir(input_dir):
    if filename.endswith(".wav") or filename.endswith(".mp3"):  # Assuming audio files are in .wav or .mp3 format
        file_path = os.path.join(input_dir, filename)
        segments, _ = model.transcribe(file_path, language="hu")  # Set language to Hungarian (hu)

        # Create the corresponding txt file
        txt_filename = filename.rsplit(".", 1)[0] + ".txt"
        txt_filepath = os.path.join(input_dir, txt_filename)

        with open(txt_filepath, "w") as f:
            for segment in segments:
                f.write(segment.text + "\n")

        print(f"Transcription for {filename} saved to {txt_filename}")

print("Transcription completed for all files.")



In [ ]:
import os
import pandas as pd
from pydub import AudioSegment

# Mappák létrehozása
os.makedirs('/content/LJSpeech/wavs', exist_ok=True)
os.makedirs('/content/LJSpeech/', exist_ok=True)

# Fájlok beolvasása
splits_path = '/content/splits'
wav_path = '/content/LJSpeech/wavs'
metadata_path = '/content/LJSpeech'

metadata = []

for root, dirs, files in os.walk(splits_path):
    for file in files:
        if file.endswith('.mp3'):
            mp3_file = os.path.join(root, file)
            txt_file = os.path.join(root, file.replace('.mp3', '.txt'))

            if os.path.exists(txt_file):
                # Olvassuk be a szöveget
                with open(txt_file, 'r', encoding='utf-8') as f:
                    text = f.read().strip()

                # MP3 konvertálása WAV formátumra
                audio = AudioSegment.from_mp3(mp3_file)
                wav_filename = f"LJ{str(len(metadata) + 1).zfill(3)}-{str(len(metadata) + 1).zfill(4)}.wav"
                wav_filepath = os.path.join(wav_path, wav_filename)
                audio.export(wav_filepath, format='wav')

                # Metadata frissítése
                metadata.append([wav_filename, text])

# Metadata.csv létrehozása
metadata_df = pd.DataFrame(metadata, columns=['file', 'text'])
metadata_df.to_csv('/content/LJSpeech/metadata.csv', index=False, header=False)

Az elkészült darabok letöltése.

In [ ]:
from google.colab import files
import shutil
import os

# A tömörítendő mappa elérési útja
folder_to_zip = '/content/LJSpeech'
# A tömörített fájl neve
output_filename = 'LJSpeech.zip'

# Tömörítés
shutil.make_archive(output_filename.replace('.zip', ''), 'zip', folder_to_zip)

# Ellenőrizzük, hogy a fájl létezik-e
if os.path.exists(output_filename):
    print(f'{output_filename} sikeresen létrehozva.')

# Letöltés
files.download(output_filename)

Munbkakörnyezet takarítása!

In [ ]:
import shutil
import os

# A törlendő mappák és fájlok elérési útjai
folders_to_delete = ['/content/files', '/content/splits', '/content/LJSpeech']
file_to_delete = '/content/splits.zip'

# Mappák törlése
for folder in folders_to_delete:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f'{folder} sikeresen törölve.')
    else:
        print(f'{folder} nem található.')

# Fájl törlése
if os.path.exists(file_to_delete):
    os.remove(file_to_delete)
    print(f'{file_to_delete} sikeresen törölve.')
else:
    print(f'{file_to_delete} nem található.')